# Langchain Intro

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from pprint import pprint
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.llms import OpenAI

### Using A Basic OpenAI Langchain LLM

- Without using a chain.
- Very simplistic.

<br>

[![image.png](https://i.postimg.cc/cLyVwqrs/image.png)](https://postimg.cc/Mv5PJF74)


In [3]:
_ = load_dotenv(find_dotenv())  # read local .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_GPT_MODEL: str = "gpt-3.5-turbo-instruct"
TEMPERATURE: float = 0.5

In [4]:
# Create LLM
llm: OpenAI = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    model=OPENAI_GPT_MODEL,
    temperature=TEMPERATURE,
)
prompt: str = "Write a very short poem."
result = llm(prompt=prompt)

print(result)

"Sun shines bright,
Birds take flight,
Nature's delight,
Pure and light."

## Updated Logic

- Add Chains (from langchain)

<br>

[![image.png](https://i.postimg.cc/RVLp2WgH/image.png)](https://postimg.cc/0brCwyRk)

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### LLM Chain

[![image.png](https://i.postimg.cc/05VBzdbB/image.png)](https://postimg.cc/0rJf3SGY)

In [6]:
# Used for text completion LLMs
prompt_template: str = """Write a very short {language} function that will {task}"""
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=prompt_template
)
code_chain: LLMChain = LLMChain(llm=llm, prompt=code_prompt)

# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = code_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'text': '\n\ndef generate_list(n):\n    return list(range(n))'
}

In [7]:
print(result.get("text"))

def generate_list(n):
    return list(range(n))

## Connecting Chains Together

- Chains in series (sequential chain)

<br>

[![image.png](https://i.postimg.cc/8Cdgm9wV/image.png)](https://postimg.cc/GBpfcMZM)

In [8]:
from langchain.chains import SequentialChain

# Used for text completion LLMs
code_prompt_template: str = (
    """Write a very short {language} function that will {task}"""
)
unittest_prompt_template: str = (
    """Write a unit test for the following {language} code: \n{code}"""
)

# Propmt(s)
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=code_prompt_template
)
unittest_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "code"], template=unittest_prompt_template
)

# Chain(s)
code_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code",  # from text to code
)
unittest_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=unittest_prompt,
    output_key="test",  # from text to test
)
# Feed the output of code_chain into unittest_chain
final_chain: SequentialChain = SequentialChain(
    chains=[code_chain, unittest_chain],
    input_variables=["language", "task"],
    output_variables=["code", "test"],
)
# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = final_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'code': '\n\ndef generate_list(start, end):\n    return list(range(start, end+1))',
    'test': '\n\nimport unittest\n\nclass TestGenerateList(unittest.TestCase):\n    \n    def 
test_generate_list(self):\n        # Test list generation with positive numbers\n        
self.assertEqual(generate_list(1,5), [1,2,3,4,5])\n        \n        # Test list generation with negative numbers\n
self.assertEqual(generate_list(-5,0), [-5,-4,-3,-2,-1,0])\n        \n        # Test list generation with start and 
end being the same number\n        self.assertEqual(generate_list(1,1), [1])\n        \n        # Test list 
generation with start and end being negative numbers\n        self.assertEqual(generate_list(-10,-5), 
[-10,-9,-8,-7,-6,-5])\n        \n        # Test list generation with start and end being decimal numbers\n        
self.assertEqual(generate_list(1.5,5.5), [1.5,2.5,3.5,4.5,5.5])\n        \n        # Test list generation with 
start and end being non-integer numbers\n        self.assertEqual(generate_list(1.5,3.5), [1.5,2.5,3.5])\n        
\n        # Test list generation with start and end being'
}

In [9]:
# Output of the unittest_chain (key="test")
print(result.get("test"))

import unittest

class TestGenerateList(unittest.TestCase):
    
    def test_generate_list(self):
        # Test list generation with positive numbers
        self.assertEqual(generate_list(1,5), [1,2,3,4,5])
        
        # Test list generation with negative numbers
        self.assertEqual(generate_list(-5,0), [-5,-4,-3,-2,-1,0])
        
        # Test list generation with start and end being the same number
        self.assertEqual(generate_list(1,1), [1])
        
        # Test list generation with start and end being negative numbers
        self.assertEqual(generate_list(-10,-5), [-10,-9,-8,-7,-6,-5])
        
        # Test list generation with start and end being decimal numbers
        self.assertEqual(generate_list(1.5,5.5), [1.5,2.5,3.5,4.5,5.5])
        
        # Test list generation with start and end being non-integer numbers
        self.assertEqual(generate_list(1.5,3.5), [1.5,2.5,3.5])
        
        # Test list generation with start and end being

<hr><br>

# Simple Terminal Chatbot

<br>

### Chat Model

- It's optimized for conversations.
- It has the following inputs:
  - System Message
  - Human Message
  - AI/Assistant Message
  
[![image.png](https://i.postimg.cc/FRgxXmJ6/image.png)](https://postimg.cc/YG0FQ5RN)

<br>

### ChatPrompt Template

[![image.png](https://i.postimg.cc/s2nTM32Z/image.png)](https://postimg.cc/4K96MkVJ)

In [10]:
dd

NameError: name 'dd' is not defined